### Практическая работа к уроку №6

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

#### Prepare data

In [105]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [8]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

In [9]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [11]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

def prepare_data(VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS):
    # column processing
    item_features.columns = [col.lower() for col in item_features.columns]
    user_features.columns = [col.lower() for col in user_features.columns]

    item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
    user_features.rename(columns={'household_key': USER_COL }, inplace=True)
    
    # берем данные для тренировки matching модели
    data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

    # берем данные для валидации matching модели
    data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                          (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


    # берем данные для тренировки ranking модели
    data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

    # берем данные для теста ranking, matching модели
    data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]
    
    print_stats_data(data_train_matcher,'train_matcher')
    print_stats_data(data_val_matcher,'val_matcher')
    print_stats_data(data_train_ranker,'train_ranker')
    print_stats_data(data_val_ranker,'val_ranker')
    
    # prefilter items: top-5000
    print('\n prefilter items in data_train_matcher:')
    n_items_before = data_train_matcher['item_id'].nunique()
    data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)
    n_items_after = data_train_matcher['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))
    
    # ищем общих пользователей
    print('\nMake cold-start to warm-start')
    common_users = data_train_matcher.user_id.values

    data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
    data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
    data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

    print_stats_data(data_train_matcher,'train_matcher')
    print_stats_data(data_val_matcher,'val_matcher')
    print_stats_data(data_train_ranker,'train_ranker')
    print_stats_data(data_val_ranker,'val_ranker')
    
    return data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker

In [12]:
VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

(data_train_matcher, data_val_matcher,
 data_train_ranker, data_val_ranker) = prepare_data(VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS)

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329

 prefilter items in data_train_matcher:


/home/mayo889/Documents/GeekBrains/GeekBrains/recommendations/06_lesson/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001

Make cold-start to warm-start
train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


# Init/train recommender

In [16]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

In [106]:
ACTUAL_COL = 'actual'

result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [107]:
def calc_recall(df_data, col_name, top_k):
    return df_data.apply(lambda row: recall_at_k(row[col_name],
                                                 row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def calc_precision(df_data, col_name, top_k):
    return df_data.apply(lambda row: precision_at_k(row[col_name],
                                                    row[ACTUAL_COL], k=top_k), axis=1).mean()

In [108]:
def base_predict(df_result, target_col_name, result_col_name, recommend_model, TOPK_RECALL, TOPK_PRECISION):
    time_start = time.time()
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=TOPK_RECALL))
    predict_time = time.time() - time_start
    
    # Сразу подсчитываем recall_at_k и precision_at_k (для сравнения со 2 уровнем модели)
    recall = calc_recall(df_result, result_col_name, TOPK_RECALL)
    precision = calc_recall(df_result, result_col_name, TOPK_PRECISION)  
    
    return df_result, recall, precision, predict_time

In [109]:
all_recommend_models = {
    'own_rec': recommender.get_own_recommendations,
    'sim_item_rec': recommender.get_similar_items_recommendation,
    'als_rec': recommender.get_als_recommendations
}

TOPK_RECALL = 50
TOPK_PRECISION = 5
result_dct = {}

for result_col_name, recommend_model in all_recommend_models.items():
    print(f"Processing {result_col_name}...")
    result_eval_matcher, recall, precision, predict_time = base_predict(result_eval_matcher,
                                                                                  USER_COL,
                                                                                  result_col_name,
                                                                                  recommend_model,
                                                                                  TOPK_RECALL,
                                                                                  TOPK_PRECISION)
    result_dct[result_col_name] = [recall, precision, predict_time]

Processing own_rec...
Processing sim_item_rec...
Processing als_rec...


In [113]:
result_train_recommender = pd.DataFrame.from_dict(result_dct, orient='index',
                                                 columns=['recall@50', 'precision@5', 'time, sec'])
result_train_recommender.sort_values(by='recall@50', ascending=False, inplace=True)
result_train_recommender

,recall@50,precision@5,"time, sec"
own_rec,0.065257,0.018202,9.390431
als_rec,0.047518,0.012348,9.644044
sim_item_rec,0.032989,0.005286,19.389914


own recommendtions + top-popular лучший recall

**Задание 1.**

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  

In [114]:
result_eval_matcher_2 = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher_2.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher_2.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [115]:
TOPK_RECALL_LIST = [20, 50, 100, 200, 500]
recommend_model = recommender.get_own_recommendations
result_dct = {}

for TOPK_RECALL in TOPK_RECALL_LIST:
    result_col_name = f"own_rec_k={TOPK_RECALL}"
    print(f"Processing {result_col_name}...")
    
    result_eval_matcher_2, recall, precision, predict_time = base_predict(result_eval_matcher_2,
                                                                          USER_COL,
                                                                          result_col_name,
                                                                          recommend_model,
                                                                          TOPK_RECALL,
                                                                          TOPK_PRECISION)
    result_dct[result_col_name] = [recall, precision, predict_time]

Processing own_rec_k=20...
Processing own_rec_k=50...
Processing own_rec_k=100...
Processing own_rec_k=200...
Processing own_rec_k=500...


In [117]:
result_own_rec = pd.DataFrame.from_dict(result_dct, orient='index',
                                        columns=['recall@k', 'precision@5', 'time, sec'])
result_own_rec.sort_values(by='recall@k', ascending=False, inplace=True)
result_own_rec

,recall@k,precision@5,"time, sec"
own_rec_k=500,0.182053,0.018202,9.739938
own_rec_k=200,0.135373,0.018202,10.896492
own_rec_k=100,0.096045,0.018202,9.790130
own_rec_k=50,0.065257,0.018202,9.297297
own_rec_k=20,0.039284,0.018202,9.018145


C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

При росте k увеличивается recall@k, но повышать k бесконечно нельзя, так как чем больше k, тем дольше нам придется обучать модель 2 уровня и большие вычислительные мощности нам придется использовать.

В нашем случае вполне можно остановиться на значении k=500, так как разница в recall существенна

### Ranking part
### Обучение модели 2 уровня

In [133]:
def prepare_rank_dataset(N_PREDICT):
    # Negativ examples with own recommendations
    
    # взяли пользователей из трейна для ранжирования
    df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
    df_match_candidates.columns = [USER_COL]
    
    # собираем кандитатов с первого этапа (matcher)
    df_match_candidates['candidates'] = df_match_candidates[USER_COL]\
                        .apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
    
    df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1)\
                                  .stack().reset_index(level=1, drop=True)
    df_items.name = 'item_id'
    df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)
    
    print_stats_data(df_match_candidates, 'Negativ examples')
    
    # Positive examples with real purchases
    df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
    df_ranker_train['target'] = 1  # тут только покупки 
    
    print_stats_data(df_ranker_train, 'Positive examples')
    
    # Merge positive and negative examples
    df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

    # чистим дубликаты
    df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])
    df_ranker_train['target'].fillna(0, inplace= True)
    print_stats_data(df_ranker_train, 'Rank dataset')
    
    return df_ranker_train

In [228]:
N_PREDICT = 50
df_ranker_train = prepare_rank_dataset(N_PREDICT)
df_ranker_train.head()

Negativ examples
Shape: (107550, 2) Users: 2151 Items: 4574
Positive examples
Shape: (169615, 3) Users: 2151 Items: 27644
Rank dataset
Shape: (106972, 3) Users: 2151 Items: 4574


,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0
2,2070,879194,0.0
3,2070,948640,0.0
4,2070,928263,0.0


### Prepare features

In [229]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [230]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [231]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [243]:
features_train_ranker = data_train_ranker.copy()

features_train_ranker = features_train_ranker.merge(item_features, on='item_id', how='left')
features_train_ranker = features_train_ranker.merge(user_features, on='user_id', how='left')

features_train_ranker.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,...,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,...,EGGS,EGGS - LARGE,1 DZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,...,CANNED JUICES,FRUIT DRINKS: CANNED & GLASS (,128 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,...,FRZN VEGETABLE/VEG DSH,FRZN BAGGED VEGETABLES - PLAIN,16 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,...,BACON,ECONOMY,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
features_train_ranker.columns.tolist()

['user_id',
 'basket_id',
 'day',
 'item_id',
 'quantity',
 'sales_value',
 'store_id',
 'retail_disc',
 'trans_time',
 'week_no',
 'coupon_disc',
 'coupon_match_disc',
 'manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [265]:
# Средний чек
add_user_features = pd.DataFrame(features_train_ranker.groupby('user_id')['sales_value'].median()).reset_index()
add_user_features.rename(columns={'sales_value': 'avg_sales_value'}, inplace=True)

add_user_features.head()

,user_id,avg_sales_value
0,1,2.290
1,2,2.000
2,4,3.315
3,6,2.495
4,7,1.595


In [263]:
departments = features_train_ranker['department'].unique().tolist()

In [271]:
user_department_price = pd.DataFrame(features_train_ranker.groupby(['user_id', 'department'])['sales_value'].median())
user_department_price.head()

sales_value
user_id department             
1                         0.000
        DELI              2.395
        DRUG GM           2.500
        GROCERY           2.290
        MEAT-PCKGD        2.815

In [273]:
user_department_price['user_id']

KeyError: 'user_id'

In [241]:
add_features_users = data_train_ranker[['user_id']].reset_index(drop=True)
add_features_users

,user_id
0,2070
1,2021
2,2021
3,2021
4,2021
...,...
169610,222
169611,462
169612,462
169613,462


In [ ]:
data_train_ranker.groupby(['user_id', )['sales_value'].median()

In [236]:
# Средний чек
data_train_ranker.groupby('user_id')['sales_value'].median()

user_id
1       2.290
2       2.000
4       3.315
6       2.495
7       1.595
        ...  
2496    1.990
2497    2.385
2498    2.500
2499    2.095
2500    2.000
Name: sales_value, Length: 2151, dtype: float64

In [217]:
"""
В полученном датасете нет информации о стоимости купленного товара
Возьмем данные о всех покупках, не затрагивающие data_val_ranker
Так как у товаров изменяется цена во времени, то возьмем медиану цен
Полученные цены также смержим в датасет для обучения

По каким-то причинам price у цен в датасете часто не указан,
но для пропусков пересчитаем его, как sales_value / quantity
"""
item_features_2 = pd.concat([data_train_matcher, data_train_ranker])
item_features_2['price'] = item_features_2['sales_value'] / item_features_2['quantity']
item_prices = pd.DataFrame(item_features_2.groupby(['item_id'])['price'].median()).reset_index()
item_prices.head()

,item_id,price
0,28116,0.33
1,28117,0.34
2,28143,0.33
3,28186,0.79
4,28304,0.53


In [226]:
data_train_ranker

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0
2107469,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,0.0,0.0
2107470,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,0.0,0.0
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2282320,222,41297772783,635,1120741,1,0.59,304,0.00,1716,91,0.0,0.0
2282321,462,41297773713,635,993339,1,1.99,304,0.00,2040,91,0.0,0.0
2282322,462,41297773713,635,995242,1,1.00,304,-0.89,2040,91,0.0,0.0
2282323,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0


In [218]:
df_ranker_train = df_ranker_train.merge(item_prices, on='item_id', how='left')
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,price
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,3.99
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,11.99
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,7.27
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6.59
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,7.99


In [224]:
user_features

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16
...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498


In [227]:
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0
